In [1]:
from datasets import load_dataset
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,4'
    
dataset = load_dataset('/data/home/wangys/DataSelection-IF/datasets/events_classification_biotech',trust_remote_code=True) 
    
classes = [class_ for class_ in dataset['train'].features['label 1'].names if class_]
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}


In [3]:
import torch
torch.save(dataset,'../datasets/events_classification.pkl')

In [2]:
from transformers import AutoTokenizer

model_path = '/data/home/wangys/model/deberta-v3-base'

tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
import pandas as pd
train = pd.read_csv('../datasets/events_classification_biotech/train.csv')


In [3]:
def preprocess_function(example):
   text = f"{example['title']}.\n{example['content']}"
#    print(example['all_labels'])
   if isinstance(example['all_labels'],str):
       all_labels = example['all_labels'].split(', ')
   else:
       all_labels = example['all_labels'] 
   labels = [0. for i in range(len(classes))]
   for label in all_labels:
       label_id = class2id[label]
       labels[label_id] = 1.
  
   example = tokenizer(text, truncation=True)
   example['labels'] = labels
   return example

tokenized_dataset = dataset.map(preprocess_function)


In [4]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["../../evaluate-main/metrics/accuracy", "../../evaluate-main/metrics/f1", "../../evaluate-main/metrics/precision", "../../evaluate-main/metrics/recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, 
                              references=labels.astype(int).reshape(-1))


In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

model = AutoModelForSequenceClassification.from_pretrained(

   '../../model/ModernBERT-base/', num_labels=len(classes),
           id2label=id2class, label2id=class2id,
                       problem_type = "multi_label_classification")

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at ../../model/ModernBERT-base/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

model = AutoModelForSequenceClassification.from_pretrained(

   model_path, num_labels=len(classes),
           id2label=id2class, label2id=class2id,
                       problem_type = "multi_label_classification")
peft_config = LoraConfig(
    task_type=TaskType.SEQUENCE_CLASSIFICATION,
    inference_mode=False,  # 设置为False用于训练模式，训练时可更新LoRA参数
    r=8,  # LoRA的秩，可根据实际情况调整
    lora_alpha=16,  # 可以调整，影响训练过程
    lora_dropout=0.1,  # 可以调整，一般是较小的值，如0.1等
    target_modules=["query", "value"]  # 要应用LoRA的模块，需根据模型结构调整，比如针对DeBERTa的相关注意力模块等
)

# 应用LoRA配置到模型上
model = get_peft_model(model, peft_config)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /data/home/wangys/model/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: type object 'TaskType' has no attribute 'SEQUENCE_CLASSIFICATION'

In [6]:
training_args = TrainingArguments(

   output_dir="my_awesome_model",
   learning_rate=2e-5,
   per_device_train_batch_size=3,
   per_device_eval_batch_size=3,
   num_train_epochs=2,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   report_to='none'
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,	
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()


/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3316013/3491357845.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


[2025-01-07 17:47:54,564] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd
/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/wangys/anaconda3/envs/deepspeed/li

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.172532,0.945063,0.257038,0.652174,0.160061
2,No log,0.160140,0.946330,0.316032,0.649289,0.208841


/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/wangys/anaconda3/envs/deepspeed/lib/pyt

TrainOutput(global_step=460, training_loss=0.2164324553116508, metrics={'train_runtime': 203.607, 'train_samples_per_second': 27.101, 'train_steps_per_second': 2.259, 'total_flos': 3656743226849076.0, 'train_loss': 0.2164324553116508, 'epoch': 2.0})